In [1]:
#!pip install split-folders

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\paxpm8\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
#import split folders
import splitfolders

In [3]:
#split the data we have from the data folder (covid and non_covid)
#ratio = (train, validation, test)
splitfolders.ratio(input="./data/", output="./output", seed=2021,
ratio=(0.7,0.15,0.15))

In [26]:
#import libraries
import keras
import tensorflow
import matplotlib.pyplot as plt
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications import densenet
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.metrics import confusion_matrix, classification_report

import numpy as np

In [13]:
#use transfer learning with DenseNet169

base_model= densenet.DenseNet169(weights="imagenet", include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)

preds = Dense(2, activation="sigmoid")(x)

model= Model(inputs= base_model.input, outputs=preds)

51888128/51877672 [==============================] - 6s 0us/step


In [14]:
# declare batch size
BATCH_SIZE = 16

In [15]:
# imagedatagenerator objects

train_datagen = ImageDataGenerator(rescale=1./255,
shear_range=0.2,
zoom_range=0.2,
brightness_range=[0.2,1.0],
horizontal_flip=True)


test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
# create train generators

train_generator = train_datagen.flow_from_directory("./output/train",
                                                    target_size=(480, 480),
                                                    color_mode="rgb",
                                                    batch_size=BATCH_SIZE,
                                                    class_mode="categorical",
                                                    shuffle=True)


Found 521 images belonging to 2 classes.


In [17]:
# create val generators

val_generator = train_datagen.flow_from_directory("./output/val",
                                                    target_size=(480, 480),
                                                    color_mode="rgb",
                                                    batch_size=BATCH_SIZE,
                                                    class_mode="categorical",
                                                    shuffle=True)


Found 111 images belonging to 2 classes.


In [18]:
#test generator
#do not shuffle

test_generator = test_datagen.flow_from_directory("./output/test",
shuffle=False)

Found 114 images belonging to 2 classes.


In [20]:
#declare callbacks for our model training

es= EarlyStopping(monitor="val_loss", patience=3) #earlystop
mc = ModelCheckpoint("./best_model.h5",
                        save_best_only=True) #checkopoint

In [27]:
#compile model for training

model.compile(optimizer= Adam(learning_rate=0.0001), loss= "binary_crossentropy", metrics=["accuracy"])



In [29]:
#train model

history = model.fit(train_generator, steps_per_epoch=train_generator.samples // BATCH_SIZE,
                    validation_data=val_generator,
                    validation_steps=val_generator.samples // BATCH_SIZE,
                    epochs=50,
                    callbacks=[es, mc])


Epoch 1/50
